In [38]:
from os import listdir
from itertools import combinations
import random
import pandas as pd
from mtcnn.mtcnn import MTCNN
import cv2
detector = MTCNN()

In [39]:
def save_face(filename, out_name, required_size=(160, 160)):
    image = cv2.imread(filename)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = detector.detect_faces(image)
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = image[y1:y2, x1:x2]
    face = cv2.resize(face, required_size)
    face = cv2.cvtColor(face, cv2.COLOR_RGB2BGR)
    cv2.imwrite(out_name, face)

In [42]:
root = 'images/'
persons = listdir(root)
completed = listdir('data/')
pos_images = []
neg_images = []
for i, person in enumerate(persons):
    path = root+person
    image_names = listdir(path)
    read_image_paths = [path + '/' + item for item in image_names]
    save_image_paths = ['data/' + item for item in image_names]
    for read, write in zip(read_image_paths, save_image_paths):
        if write in completed: continue
        save_face(read, write)

In [44]:
root = 'images/'
persons = listdir(root)
pos_images = []
neg_images = []
for i, person in enumerate(persons):
    path = root+person
    image_names = listdir(path) # abc_0001.jpg
    if len(image_names)>1:
        read_image_paths = [path + '/' + item for item in image_names]
        save_image_paths = ['data/' + item for item in image_names]
        comb = list(combinations(save_image_paths, 2)) 
        random.shuffle(comb)
        pos_images += comb[:3]
        
        neg_idx = i
        neg_samples = 0
        while neg_samples <= 2: 
            neg_idx = random.randint(0, len(persons)-1)
            if neg_idx == i: continue
            path = root+persons[neg_idx]
            temp = listdir(path)
            temp = ['data/' + item for item in temp][random.randint(0, len(temp)-1)]
            neg_images.append((comb[0][0], temp))
            neg_samples+=1
        

In [45]:
random.shuffle(pos_images)
random.shuffle(neg_images)
neg_images[:10]

[('data/Fernando_Vargas_0003.jpg', 'data/Adam_Freier_0001.jpg'),
 ('data/Eric_Hinske_0001.jpg', 'data/Michelangelo_Antonioni_0001.jpg'),
 ('data/Tom_Jones_0001.jpg', 'data/Jim_OBrien_0003.jpg'),
 ('data/Naomi_Watts_0001.jpg', 'data/Don_King_0001.jpg'),
 ('data/Robert_Fico_0001.jpg', 'data/Will_Smith_0002.jpg'),
 ('data/Paradorn_Srichaphan_0003.jpg', 'data/Gene_Orza_0001.jpg'),
 ('data/Stephen_Ambrose_0001.jpg', 'data/Mark_Rosenbaum_0001.jpg'),
 ('data/Azra_Akin_0002.jpg', 'data/Anthony_Rackauckas_0001.jpg'),
 ('data/Sergey_Lavrov_0004.jpg', 'data/Willie_Wilson_0001.jpg'),
 ('data/Allyson_Felix_0001.jpg', 'data/Augustin_Calleri_0001.jpg')]

In [46]:
df = pd.DataFrame()
image1 = []
image2 = []
label = []

for pair in pos_images[:1000]:
    image1.append(pair[0])
    image2.append(pair[1])
    label.append(1)
    
for pair in neg_images[:1000]:
    image1.append(pair[0])
    image2.append(pair[1])
    label.append(0)
    
df['image1'] = image1
df['image2'] = image2
df['label'] = label
df.head()

,image1,image2,label
0,data/Billy_Crystal_0001.jpg,data/Billy_Crystal_0003.jpg,1
1,data/Britney_Spears_0008.jpg,data/Britney_Spears_0014.jpg,1
2,data/Joseph_Deiss_0002.jpg,data/Joseph_Deiss_0003.jpg,1
3,data/Barbra_Streisand_0001.jpg,data/Barbra_Streisand_0002.jpg,1
4,data/Rebekah_Chantay_Revels_0001.jpg,data/Rebekah_Chantay_Revels_0002.jpg,1


In [47]:
df.to_csv('data.csv', index=False)